## Illustrative example implementing COSMOS ideas on Korda2020 optimisation problem

In the paper by Korda2020 we are solving 
$$ \min_{\lambda_i, g_i} \left\|
  \begin{bmatrix}
      h_i(x^1(0)) & h_i(x^2(0)) & \cdots \\ 
      h_i(x^1(1)) & h_i(x^2(1)) & \cdots \\ 
      h_i(x^1(2)) & h_i(x^2(2)) & \cdots \\ 
      \vdots & \vdots & \ddots %\\ h_i(x^1(N)) \\ h_i(x^2(0)) \\ \vdots
  \end{bmatrix}- 
  \begin{bmatrix}
      1 & 1 & 1 & \cdots \\
      \lambda_{1} & \lambda_{2} & \lambda_{3} & \cdots \\
      \lambda_{1}^2 & \lambda_{2}^2 & \lambda_{3}^2 & \cdots \\
      \vdots & \vdots & \vdots & \ddots
  \end{bmatrix}
  \begin{bmatrix}% End of phantom section for vertical brace alignment
      g_{1} (x^1(0)) & g_{1} (x^2(0)) & \cdots \\ 
      g_{2} (x^1(0)) & g_{2} (x^2(0)) & \cdots \\
      \vdots & \vdots & \ddots
  \end{bmatrix} \right\| $$

That is, given a matrix $Y$ we seek a decomposition into a product of a Vandermonde matrix $\Lambda$ and a matrix of initial states $G$, formulated as a least-squares problem $\| Y - \Lambda G \|$. The idea is to turn this problem from a nonconvex problem into a convex problem with rank constraint, and apply the ideas in COSMOS to this work. 
Defining $D_\lambda = \text{diag}(\lambda_1, \lambda_2, \dots)$, and $\overline \Lambda$ as the matrix $\Lambda$, but shifted one place up, we obtain that $\overline \Lambda = \Lambda D_{\lambda}$. These relationships can be written in terms of a rank constraint, for which we introduce
$$ H = \begin{bmatrix} \hat M & \Lambda & \overline \Lambda \\ \hat G & I_{N_g} & D_\lambda \end{bmatrix}. $$

By the Schur decomposition, the rank of the matrix $\hat M - \Lambda I_{N_g} \hat G$ is equivalent to rank of the left $2 \times 2$ block submatrix of $H$. Similarly, the introduced relationship between $\Lambda$ and its shifted matrix $\overline \Lambda$, gives the latter columns the same rank. Hence we obtain the optimisation problem
$$ \min \| Y - \hat M \| $$
subject to $\text{rank}(H) = N_g$


The code below is an iterative method, solving 
$$ \text{argmin}_{\hat M, \hat G, \Lambda, \overline \Lambda, \lambda_i} \| Y - \hat M \|_2 + \rho \left( \| H \|_* - \text{tr} \left( U_1^T H V_1 \right) \right) $$
subject to: 
\\[\Lambda[0,:] = [1, 1, \dots] \newline
  \overline \Lambda[0:end-1, :] = \Lambda[1:,:] \newline
  \Lambda[1,:] = [\lambda_1, \lambda_2, \dots]\\]
iteratively, using the truncated singular value decomposition for $H = U S V^*$. 

In [12]:
import cvxpy as cvx
import numpy as np
np.random.seed(123)

def H_Theta(M, Theta, ThetaShift, X, Id, L):
    '''Function for constructing the blockmatrix in the rank constraint optimisation problem
    Using the input matrices, the following block matrix is constructed:
    | M  Theta  ThetaShift |
    | X    Id     L        |
    '''
    return cvx.bmat([[M, Theta, ThetaShift], [X, Id, L]])


N = 5                             # Number of rows of measurement data Y, i.e. measurement sequence length
N_t = 100                           # Number of columns of measurement data Y, i.e. number of measurement sequences
N_g = 3                            # Rank used as rank constraint, and to construct measurement data Y
Id = np.eye(N_g)                   # Usefull identity matrix
# Total number of optimisation variables = N_t * N_g + N_g
# Total number of constrained variables = N_g + N_g

# Define Y as decomposed product of a VanderMonde matrix and random initial states.
Y = np.vander([0.9, 0.95, 0.98], N, increasing=True).T @ np.random.rand(N_g, N_t) 

# Initialise U and V as empty matrices
U_j = [np.zeros((N + N_g, N_g))]
V_j = [np.zeros((N_t + 2 * N_g, N_g))]

# Initialise CVX optimisation problem
Mhat = cvx.Variable(Y.shape)
Ghat = cvx.Variable((N_g, N_t))
ThetaHat = cvx.Variable((N,N_g))
ThetaHatShift = cvx.Variable((N, N_g))
Lhat = cvx.diag(cvx.Variable(N_g, complex=False))      

# Constraints: 
#        Requirement that first row of Theta[0,:] = [ 1, 1, 1, ...]
#        Requirement that second row of Theta[1,:] = ThetaShift[0,:]
#        Requirement that second row of Theta[1,:] = [lambda_1, lambda_2, ...]
cons = [ThetaHat[0,:] == np.ones((N_g,)), ThetaHat[1,:] == ThetaHatShift[0,:]] #, ThetaHat[1,:] == cvx.diag(Lhat)]

# Construct rank constraint matrix
Fullmatrix = H_Theta(Mhat, ThetaHat, ThetaHatShift, Ghat, Id, Lhat)

# Initialise variables that change on optimisation iteration
U = cvx.Parameter((N + N_g, N_g), complex=False)
V = cvx.Parameter((N_t + 2 * N_g, N_g), complex=False)
rho = cvx.Parameter(pos=True)

# Define objective function and problem handle
obj = cvx.norm(Y - Mhat,2) +  rho * ( cvx.norm(Fullmatrix, "nuc") - (cvx.trace(U.T @ Fullmatrix @ V))) 
prob = cvx.Problem(cvx.Minimize(obj), cons)

# Initialise optimisation iteration
i = 0
eps = 1
rho_new = 0.01       # Initial value for rho
mu = 1.4            # Growth factor for rho
rho_max = 10     # Maximum value for rho
prev_value = 1e5

# Iterate while error is large and number of iterations is small enough
while eps > 1e-4 and i < 10:
    rho_new = min(mu * rho_new, rho_max)
    
    # Set CVX problem parameters
    rho.value = rho_new
    U.value = U_j[i]
    V.value = V_j[i]
    
    # Solve optimisation problem iteration using cvx
    prob.solve(verbose=False)

    # Compute SVD of rank-constraint matrix value
    Res = Fullmatrix.value # H_Theta_num(M, Theta, ThetaShift, X, Id, L)
    U1, s, V1 = np.linalg.svd(Res, full_matrices=True)

    U_j.append(U1[:,:N_g])
    V_j.append(V1[:N_g,:].T)
 
    # Update error value, and print update
    eps = abs((obj.value - prev_value) / obj.value)
    prev_value = obj.value
    i = i + 1
    print(f'Iteration {i} completed with error {obj.value:.3f} and rho={rho_new:.2f}')
    
# Extract solutions from optimisation procedure
M = Mhat.value
G = Ghat.value
Theta = ThetaHat.value
ThetaShift = ThetaHatShift.value
L = Lhat.value 

print()
print(f'------------------ Finished optimisation procedure with status word: {prob.status} ----------------------------')
print(f'Found eigenvalue sequence                                  {np.array2string(np.diag(L), precision=2)}')
# print(Theta)
# print(ThetaShift)
print(f"Final nuclear norm of rank-constraint matrix               {np.linalg.norm(Res, 'nuc'):.3f}")
print(f'Least-square norm error ||Y - M||                          {np.linalg.norm(Y - M, 2):.3f}')
print(f'Least-square norm error ||Y - L*G||                        {np.linalg.norm(Y - np.vander(np.diag(L), N, increasing=True).T @ G, 2):.3f}') 
# print(Res)

Iteration 1 completed with error 0.494 and rho=0.01
Iteration 2 completed with error 0.020 and rho=0.02
Iteration 3 completed with error 0.028 and rho=0.03
Iteration 4 completed with error 0.038 and rho=0.04
Iteration 5 completed with error 0.052 and rho=0.05
Iteration 6 completed with error 0.063 and rho=0.08
Iteration 7 completed with error 0.065 and rho=0.11
Iteration 8 completed with error 0.063 and rho=0.15
Iteration 9 completed with error 0.046 and rho=0.21
Iteration 10 completed with error 0.008 and rho=0.29

------------------ Finished optimisation procedure with status word: optimal ----------------------------
Found eigenvalue sequence                                  [0.47 1.08 0.9 ]
Final nuclear norm of rank-constraint matrix               36.728
Least-square norm error ||Y - M||                          0.000
Least-square norm error ||Y - L*G||                        2.201


In [13]:
print(np.vander(np.diag(L), N, increasing=True).T @ G)
print(Y - np.vander(np.diag(L), N, increasing=True).T @ G)
# print(Y - M)

[[1.75223734 1.01947475 0.98615752 1.67836213 1.81088774 1.65841538
  1.92986295 1.93792728 1.99650621 0.82602992 1.45228522 1.60205586
  1.03554769 1.67691584 1.14828596 1.77139032 0.94261905 1.01826443
  1.37984602 1.70699333 1.95697824 2.04617613 1.25356295 2.04788265
  2.54272747 1.39778601 1.08299572 1.29480607 1.41428312 1.07267625
  1.06478893 0.69031032 1.26977319 2.1673012  2.30427615 0.43486081
  1.156632   2.0785908  1.93432802 1.67590177 0.72285108 0.89738219
  2.23314939 1.35898726 1.42075851 1.46583305 2.18788415 1.37603951
  2.02708674 2.25095484 0.4948115  2.05067961 1.60523365 1.26207203
  1.19897842 1.38305953 1.73261853 2.43458933 1.59697651 1.21764442
  1.60960915 1.41842334 2.18359428 0.99609719 1.79930142 1.11196345
  1.20620371 1.40363192 1.245482   1.78095654 1.12851619 1.32681376
  1.49251049 1.91059761 1.45979955 0.88510646 1.2360512  0.59053433
  1.18448992 1.14602258 1.76857057 0.83035945 0.882755   1.97771696
  1.43901415 2.36059088 1.33593884 1.46684344 1.